In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('/kaggle/input/comment-category-prediction-challenge/train.csv')
test = pd.read_csv('/kaggle/input/comment-category-prediction-challenge/test.csv')

def clean(text):
    if pd.isnull(text): return 'empty'
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip() or 'empty'

train['c'] = train['comment'].apply(clean)
test['c'] = test['comment'].apply(clean)

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2), sublinear_tf=True)
tfidf.fit(pd.concat([train['c'], test['c']]))
X_train = tfidf.transform(train['c'])
X_test = tfidf.transform(test['c'])

le = LabelEncoder()
y = le.fit_transform(train['label'])

model = LogisticRegression(C=1.0, max_iter=300, solver='lbfgs', random_state=42)
model.fit(X_train, y)

preds = le.inverse_transform(model.predict(X_test))
pd.DataFrame({'ID': test['post_id'], 'label': preds}).to_csv('/kaggle/working/submission.csv', index=False)
print("DONE! submission.csv ban gayi!")

DONE! submission.csv ban gayi!
